In [1]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns; sns.set()
sns.__version__

import pickle

In [2]:
!pip show pandas

Name: pandas
Version: 1.2.5
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: 
Author-email: 
License: BSD
Location: /home/minsu/proj/tensorflow_hub_env/lib/python3.8/site-packages
Requires: numpy, python-dateutil, pytz
Required-by: seaborn, statsmodels


In [1]:
! pip show statsmodels

Name: statsmodels
Version: 0.13.2
Summary: Statistical computations and models for Python
Home-page: https://www.statsmodels.org/
Author: 
Author-email: 
License: BSD License
Location: /home/minsu/proj/tensorflow_hub_env/lib/python3.8/site-packages
Requires: numpy, packaging, pandas, patsy, scipy
Required-by: 


## Data Preparation

In [2]:
df_paper_grant_stat= pd.read_pickle('/data/interdisciplinarity/df_paper_grant_stat_regression.pkl')

In [3]:
with open('/data/sci_of_sci/linkage/d_paper2fos-dist-in-ref.pkl', 'rb') as filename:
    d_reference = pickle.load(filename)

In [4]:
df_paper_grant_stat['mag_id'] = df_paper_grant_stat['mag_id'].apply(str)

In [5]:
df_paper_grant_stat['paper_fos'] = df_paper_grant_stat['paper_fos'].astype(int, errors='ignore').apply(str)

In [6]:
df_paper_grant_stat['num_ref'] = df_paper_grant_stat['mag_id'].map(d_reference).apply(lambda x: x if pd.isnull(x) else sum(x.values()))

In [7]:
tmp = df_paper_grant_stat[['interdisciplinarity_reference','mean_grant_interdisciplinarity','avg_dist_btw_grants',
                           'num_authors','sum_funding_usd','num_grant','num_funding_country','num_institutes',
                           'norm_c10_year_fos','c_hit_rate_5_lvl-0','c10',
                           'num_ref','pub_year','paper_fos']].dropna().copy()

In [8]:
tmp = tmp[tmp['num_grant']>1]

### OLS

In [9]:
#-https://medium.com/@kylecaron/introduction-to-linear-regression-part-2-standardization-and-regression-diagnostics-a15cb27944b1

def standardize(series):
    return (series - series.mean()) / series.std()

In [10]:
reg_df_standardized = tmp.copy()
reg_df_standardized[['norm_c10_year_fos',
                     'c10',
                     'interdisciplinarity_reference',
                     'mean_grant_interdisciplinarity',
                     'avg_dist_btw_grants',
                     'num_authors',
                     'num_grant',
                     'num_funding_country',
                     'num_institutes',
                     'sum_funding_usd',
                     'num_ref']] = reg_df_standardized[['norm_c10_year_fos',
                                                        'c10',
                                                        'interdisciplinarity_reference',
                                                        'mean_grant_interdisciplinarity',
                                                        'avg_dist_btw_grants',
                                                        'num_authors',
                                                        'num_grant',
                                                        'num_funding_country',
                                                        'num_institutes',
                                                        'sum_funding_usd',
                                                        'num_ref']].apply(standardize)

In [13]:
reg_df_standardized_w_sim = tmp.copy()
reg_df_standardized_w_sim['avg_sim_btw_grants'] = 1-reg_df_standardized_w_sim['avg_dist_btw_grants']
reg_df_standardized_w_sim[['norm_c10_year_fos',
                           'c10',
                           'interdisciplinarity_reference',
                           'mean_grant_interdisciplinarity',
                           'avg_sim_btw_grants',
                           'num_authors',
                           'num_grant',
                           'num_funding_country',
                           'num_institutes',
                           'sum_funding_usd',
                           'num_ref']] = reg_df_standardized_w_sim[['norm_c10_year_fos',
                                                                    'c10',
                                                                    'interdisciplinarity_reference',
                                                                    'mean_grant_interdisciplinarity',
                                                                    'avg_sim_btw_grants',
                                                                    'num_authors',
                                                                    'num_grant',
                                                                    'num_funding_country',
                                                                    'num_institutes',
                                                                    'sum_funding_usd',
                                                                    'num_ref']].apply(standardize)

In [249]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

reg1 = ols('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants', 
           data=reg_df_standardized).fit()
reg2 = ols('norm_c10_year_fos ~ num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes', 
           data=reg_df_standardized).fit()
reg3 = ols('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes', 
            data=reg_df_standardized).fit()
reg4 = ols('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            C(pub_year)', data=reg_df_standardized).fit()
reg5 = ols('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            num_ref', data=reg_df_standardized).fit()
reg6 = ols('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            C(pub_year) + num_ref', data=reg_df_standardized).fit()

results = summary_col([reg1, reg2, reg3, reg4, reg5, reg6], stars=True, float_format='%0.3f',
                      model_names=['Model\n(1)', 'Model\n(2)', 'Model\n(3)', 
                                   'Model\n(4)', 'Model\n(5)', 'Model\n(6)'],
                      info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                 'R2':lambda x: "{:.3f}".format(x.rsquared)})
print(results)


                                 Model    Model     Model     Model     Model     Model  
                                  (1)      (2)       (3)       (4)       (5)       (6)   
-----------------------------------------------------------------------------------------
C(pub_year)[T.1986]                                         0.019               0.026    
                                                            (0.122)             (0.121)  
C(pub_year)[T.1987]                                         0.009               -0.004   
                                                            (0.112)             (0.111)  
C(pub_year)[T.1988]                                         0.011               0.001    
                                                            (0.109)             (0.108)  
C(pub_year)[T.1989]                                         -0.001              -0.013   
                                                            (0.108)             (0.107)  
C(pub_yea

In [297]:
reg4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    c10   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     648.0
Date:                Wed, 08 Nov 2023   Prob (F-statistic):               0.00
Time:                        14:42:01   Log-Likelihood:            -7.1587e+05
No. Observations:              515796   AIC:                         1.432e+06
Df Residuals:                  515744   BIC:                         1.432e+06
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -0.2526      0.104     -2.438      0.015      -0.456      -0.050
paper_fos[T.127313418.0]           0.1129      0.012      9.643      0.000       0.090       0.136
paper_fos[T.127413603.0]           0.0852      0.019      4.406      0.000       0.047       0.123
paper_fos[T.138885662.0]           0.0749      0.292      0.256      0.798      -0.498       0.648
paper_fos[T.142362112.0]           0.3761      0.434      0.867      0.386      -0.474       1.226
paper_fos[T.144024400.0]           0.1311      0.047      2.811      0.005       0.040       0.222
paper_fos[T.144133560.0]           0.1584      0.142      1.119      0.263      -0.119       0.436
paper_fos[T.15744967.0]            0.1754      0.009     20.217      0.000       0.158       0.192
paper_fos[T.162324750.0]           0.1976      0.033      6.043      0.000       0.134       0.262
paper_fos[T.17744445.0]           -0.0101      0.217     -0.047      0.963      -0.435       0.415
paper_fos[T.185592680.0]           0.0483      0.008      6.065      0.000       0.033       0.064
paper_fos[T.192562407.0]           0.1310      0.012     10.783      0.000       0.107       0.155
paper_fos[T.205649164.0]           0.3799      0.075      5.071      0.000       0.233       0.527
paper_fos[T.33923547.0]            0.1173      0.012      9.899      0.000       0.094       0.141
paper_fos[T.39432304.0]            0.6554      0.065     10.126      0.000       0.529       0.782
paper_fos[T.41008148.0]            0.1229      0.010     12.786      0.000       0.104       0.142
paper_fos[T.71924100.0]            0.2028      0.008     26.353      0.000       0.188       0.218
paper_fos[T.86803240.0]            0.2175      0.007     29.903      0.000       0.203       0.232
paper_fos[T.95457728.0]            0.0195      0.198      0.098      0.922      -0.369       0.408
C(pub_year)[T.1986]                0.0546      0.119      0.457      0.648      -0.180       0.289
C(pub_year)[T.1987]                0.0147      0.110      0.134      0.894      -0.201       0.230
C(pub_year)[T.1988]                0.0288      0.106      0.270      0.787      -0.180       0.237
C(pub_year)[T.1989]                0.0285      0.105      0.270      0.787      -0.178       0.235
C(pub_year)[T.1990]                0.0472      0.105      0.450      0.653      -0.158       0.253
C(pub_year)[T.1991]                0.0182      0.105      0.174      0.862      -0.187       0.223
C(pub_year)[T.1992]                0.0468      0.104      0.448      0.654      -0.158       0.251
C(pub_year)[T.1993]                0.0832      0.104      0.797      0.425      -0.121       0.288
C(pub_year)[T.1994]                0.0857      0.104      0.821      0.412      -0.119       0.290
C(pub_year)[T.

In [292]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

reg1 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            paper_fos', data=reg_df_standardized).fit()
reg2 = ols('c10 ~ num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes +\
            paper_fos', data=reg_df_standardized).fit()
reg3 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes +\
            paper_fos', data=reg_df_standardized).fit()
reg4 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            paper_fos + C(pub_year)', data=reg_df_standardized).fit()
reg5 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            paper_fos + num_ref', data=reg_df_standardized).fit()
reg6 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            paper_fos + C(pub_year) + num_ref', data=reg_df_standardized).fit()

results = summary_col([reg1, reg2, reg3, reg4, reg5, reg6], stars=True, float_format='%0.3f',
                      model_names=['Model\n(1)', 'Model\n(2)', 'Model\n(3)', 
                                   'Model\n(4)', 'Model\n(5)', 'Model\n(6)'],
                      info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                 'R2':lambda x: "{:.3f}".format(x.rsquared)})
print(results)


                                 Model     Model     Model     Model     Model     Model  
                                  (1)       (2)       (3)       (4)       (5)       (6)   
------------------------------------------------------------------------------------------
C(pub_year)[T.1986]                                          0.050               0.055    
                                                             (0.121)             (0.119)  
C(pub_year)[T.1987]                                          0.032               0.015    
                                                             (0.112)             (0.110)  
C(pub_year)[T.1988]                                          0.043               0.029    
                                                             (0.108)             (0.106)  
C(pub_year)[T.1989]                                          0.044               0.028    
                                                             (0.107)             (0.105) 

In [293]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

reg1 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            paper_fos + C(pub_year)', data=reg_df_standardized).fit()
reg2 = ols('c10 ~ num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes +\
            paper_fos + C(pub_year)', data=reg_df_standardized).fit()
reg3 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes +\
            paper_fos + C(pub_year)', data=reg_df_standardized).fit()
reg4 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            paper_fos + C(pub_year) + num_ref', data=reg_df_standardized).fit()

results = summary_col([reg1, reg2, reg3, reg4], stars=True, float_format='%0.3f',
                      model_names=['Model\n(1)', 'Model\n(2)', 'Model\n(3)', 'Model\n(4)'],
                      info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                 'R2':lambda x: "{:.3f}".format(x.rsquared)})
print(results)


                                 Model     Model     Model     Model  
                                  (1)       (2)       (3)       (4)   
----------------------------------------------------------------------
C(pub_year)[T.1986]            0.050     0.063     0.050     0.055    
                               (0.122)   (0.121)   (0.121)   (0.119)  
C(pub_year)[T.1987]            0.033     0.044     0.032     0.015    
                               (0.112)   (0.112)   (0.112)   (0.110)  
C(pub_year)[T.1988]            0.049     0.054     0.043     0.029    
                               (0.109)   (0.108)   (0.108)   (0.106)  
C(pub_year)[T.1989]            0.051     0.055     0.044     0.028    
                               (0.108)   (0.107)   (0.107)   (0.105)  
C(pub_year)[T.1990]            0.072     0.074     0.064     0.047    
                               (0.107)   (0.107)   (0.107)   (0.105)  
C(pub_year)[T.1991]            0.052     0.052     0.042     0.018    
     

In [240]:
reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    c10   R-squared:                       0.056
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     926.7
Date:                Wed, 08 Nov 2023   Prob (F-statistic):               0.00
Time:                        10:42:08   Log-Likelihood:            -7.1703e+05
No. Observations:              515796   AIC:                         1.434e+06
Df Residuals:                  515762   BIC:                         1.435e+06
Df Model:                          33                                         
Covariance Type:            nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                         -0.0743      0.104     -0.717      0.473      -0.277       0.129
C(pub_year)[T.1986]                0.0482      0.120      0.403      0.687      -0.186       0.283
C(pub_year)[T.1987]                0.0147      0.110      0.134      0.893      -0.201       0.231
C(pub_year)[T.1988]                0.0304      0.107      0.285      0.776      -0.179       0.239
C(pub_year)[T.1989]                0.0328      0.106      0.311      0.756      -0.174       0.240
C(pub_year)[T.1990]                0.0525      0.105      0.499      0.618      -0.154       0.259
C(pub_year)[T.1991]                0.0260      0.105      0.248      0.804      -0.179       0.231
C(pub_year)[T.1992]                0.0552      0.105      0.527      0.598      -0.150       0.260
C(pub_year)[T.1993]                0.0915      0.105      0.874      0.382      -0.114       0.297
C(pub_year)[T.1994]                0.0949      0.105      0.907      0.364      -0.110       0.300
C(pub_year)[T.1995]                0.1250      0.104      1.198      0.231      -0.079       0.329
C(pub_year)[T.1996]                0.1649      0.104      1.582      0.114      -0.039       0.369
C(pub_year)[T.1997]                0.1605      0.104      1.542      0.123      -0.043       0.364
C(pub_year)[T.1998]                0.1436      0.104      1.381      0.167      -0.060       0.347
C(pub_year)[T.1999]                0.1242      0.104      1.196      0.232      -0.079       0.328
C(pub_year)[T.2000]                0.1210      0.104      1.166      0.244      -0.082       0.324
C(pub_year)[T.2001]                0.0995      0.104      0.959      0.338      -0.104       0.303
C(pub_year)[T.2002]                0.0981      0.104      0.945      0.345      -0.105       0.301
C(pub_year)[T.2003]                0.1019      0.104      0.983      0.326      -0.101       0.305
C(pub_year)[T.2004]                0.0919      0.104      0.886      0.376      -0.111       0.295
C(pub_year)[T.2005]                0.0744      0.104      0.717      0.473      -0.129       0.278
C(pub_year)[T.2006]                0.0521      0.104      0.502      0.616      -0.151       0.255
C(pub_year)[T.2007]                0.0582      0.104      0.561      0.575      -0.145       0.261
C(pub_year)[T.2008]                0.0543      0.104      0.524      0.600      -0.149       0.257
C(pub_year)[T.2009]                0.0327      0.104      0.315      0.753      -0.170       0.236
interdisciplinarity_reference      0.0116      0.001      8.143      0.000       0.009       0.014
mean_grant_interdisciplinarity    -0.0309      0.002    -20.392      0.000      -0.034      -0.028
avg_dist_btw_grants               -0.0058      0.001     -3.978      0.000      -0.009      -0.003
num_authors   

In [15]:
from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col

reg1 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_sim_btw_grants +\
            paper_fos + C(pub_year)', data=reg_df_standardized_w_sim).fit()
reg2 = ols('c10 ~ num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes +\
            paper_fos + C(pub_year)', data=reg_df_standardized_w_sim).fit()
reg3 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_sim_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes +\
            paper_fos + C(pub_year)', data=reg_df_standardized_w_sim).fit()
reg4 = ols('c10 ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_sim_btw_grants +\
            num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
            paper_fos + C(pub_year) + num_ref', data=reg_df_standardized_w_sim).fit()

results = summary_col([reg1, reg2, reg3, reg4], stars=True, float_format='%0.3f',
                      model_names=['Model\n(1)', 'Model\n(2)', 'Model\n(3)', 'Model\n(4)'],
                      info_dict={'N':lambda x: "{0:d}".format(int(x.nobs)),
                                 'R2':lambda x: "{:.3f}".format(x.rsquared)})
print(results)


                                 Model     Model     Model     Model  
                                  (1)       (2)       (3)       (4)   
----------------------------------------------------------------------
C(pub_year)[T.1986]            0.050     0.063     0.050     0.055    
                               (0.122)   (0.121)   (0.121)   (0.119)  
C(pub_year)[T.1987]            0.033     0.044     0.032     0.015    
                               (0.112)   (0.112)   (0.112)   (0.110)  
C(pub_year)[T.1988]            0.049     0.054     0.043     0.029    
                               (0.109)   (0.108)   (0.108)   (0.106)  
C(pub_year)[T.1989]            0.051     0.055     0.044     0.028    
                               (0.108)   (0.107)   (0.107)   (0.105)  
C(pub_year)[T.1990]            0.072     0.074     0.064     0.047    
                               (0.107)   (0.107)   (0.107)   (0.105)  
C(pub_year)[T.1991]            0.052     0.052     0.042     0.018    
     

In [219]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

y, X = dmatrices('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
                  num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
                  C(pub_year)', data=reg_df_standardized, return_type='dataframe')

vif_df = pd.DataFrame()
vif_df['variable'] = X.columns 

vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_df

,variable,VIF
0,Intercept,5862.163064
1,C(pub_year)[T.1986],3.975406
2,C(pub_year)[T.1987],8.705099
3,C(pub_year)[T.1988],17.485903
4,C(pub_year)[T.1989],25.913539
5,C(pub_year)[T.1990],33.305427
6,C(pub_year)[T.1991],42.493101
7,C(pub_year)[T.1992],47.819394
8,C(pub_year)[T.1993],49.607454
9,C(pub_year)[T.1994],53.537565


In [217]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

y, X = dmatrices('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
                  num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
                  C(pub_year) + num_ref', data=reg_df_standardized, return_type='dataframe')

vif_df = pd.DataFrame()
vif_df['variable'] = X.columns 

vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_df

,variable,VIF
0,Intercept,5862.381767
1,C(pub_year)[T.1986],3.975407
2,C(pub_year)[T.1987],8.705109
3,C(pub_year)[T.1988],17.485917
4,C(pub_year)[T.1989],25.913572
5,C(pub_year)[T.1990],33.305474
6,C(pub_year)[T.1991],42.493247
7,C(pub_year)[T.1992],47.819560
8,C(pub_year)[T.1993],49.607750
9,C(pub_year)[T.1994],53.537908


In [218]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

y, X = dmatrices('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
                  num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
                  num_ref', data=reg_df_standardized, return_type='dataframe')

vif_df = pd.DataFrame()
vif_df['variable'] = X.columns 

vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_df

,variable,VIF
0,Intercept,1.000000
1,interdisciplinarity_reference,1.096247
2,mean_grant_interdisciplinarity,1.248104
3,avg_dist_btw_grants,1.160217
4,num_authors,1.039660
5,sum_funding_usd,1.175126
6,num_grant,1.707468
7,num_funding_country,1.021926
8,num_institutes,1.585343
9,num_ref,1.017684


In [298]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

y, X = dmatrices('norm_c10_year_fos ~ interdisciplinarity_reference + mean_grant_interdisciplinarity + avg_dist_btw_grants +\
                  num_authors + sum_funding_usd + num_grant + num_funding_country + num_institutes + \
                  paper_fos + C(pub_year) + num_ref', data=reg_df_standardized, return_type='dataframe')

vif_df = pd.DataFrame()
vif_df['variable'] = X.columns 

vif_df['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

vif_df

,variable,VIF
0,Intercept,5891.322635
1,paper_fos[T.127313418.0],1.505703
2,paper_fos[T.127413603.0],1.146128
3,paper_fos[T.138885662.0],1.000551
4,paper_fos[T.142362112.0],1.000297
5,paper_fos[T.144024400.0],1.024318
6,paper_fos[T.144133560.0],1.002470
7,paper_fos[T.15744967.0],2.782259
8,paper_fos[T.162324750.0],1.049538
9,paper_fos[T.17744445.0],1.001054
